In [62]:
import pandas as pd
import datetime as dt
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import numpy as np
import time
import plotly.graph_objects as go

In [51]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\alpha_vantage_api.txt','r') as f:
    alpha_vantage_api = f.readlines()[0]

In [52]:
#TODO: perform analysis in Jupyter notebook - decide on information to query
#TODO: choose information to store in RDBMS
#TODO: define schema in RDBMS
#TODO: build airflow pipeline to extract, transform and load into RDBMS
#TODO: build dash (dash over streamlit for adaptability)
#TODO: deploy dash
#TODO: have highlighted areas in basketball court that shows top 10 players in every range of shooting based on FG%

># Request for Traditional Endpoint Data

In [103]:
url = "https://stats.nba.com/stats/leaguedashplayershotlocations?" # /leaguedashplayerstats?, /playerestimatedmetrics?, /leaguedashplayerclutch?, /leagueleaders? /playergamelogs?, /leaguedashplayershotlocations?

parameters = {
# 'AheadBehind': '', # (Only for clutch endpoint) Ahead or Behind, Behind or Tied, Ahead or Tied
# 'ActiveFlag': '', # (Only for leagueleaders endpoint) No, Yes
# 'ClutchTime': '', # (Only for clutch endpoint) Last 5 mins, Last 4 mins, Last 3 mins, Last 2 mins, Last 1 mins, Last 30 Seconds, Last 10 Seconds 
'College': '', 
'Conference': '', # East, West
'Country': '',
'DateFrom': '',
'DateTo': '',
'DistanceRange': '5ft Range', # (Only for leaguedashplayershotlocations) 5ft Range, 8ft Range, By Zone
'Division': '', # Central, Atlantic, Southeast, Southwest, Northwest, Pacific 
'DraftPick': '', # 1st Round, 2nd Round, 1st Pick, Lottery Pick, Top 5 Pick, Top 10 Pick, Top 15 Pick, Top 20 Pick, Top 25 Pick, Picks 11 Thru 20, Picks 21 Thru 30, Undrafted
'DraftYear': '',
'GameScope': '',
'GameSegment': '', # First Half, Second Half, Overtime
'Height': '', # LT 6-0, GT 6-9
'LastNGames': '0',
'LeagueID': '00',
'Location': '', # Home, Road
'MeasureType': 'Base', # Base, Advanced, Misc, Scoring, Usage, Opponent, Defense, 
'Month': '0', # January=4, February=5, ..., September=9, October=1, November=2, December=3
'OpponentTeamID': '0',
'Outcome': '', # W, L
'PORound': '0', #Conference Quarter-Finals=1, Conference Semi-Finals=2, Conference Finals=3, Finals=4
'PaceAdjust': 'N',
'PerMode': 'Totals', #Per Mode: Totals, PerGame, Per100Possessions, Per100Plays, Per48, Per40, Per36, PerMinute, PerPossession, PerPlay, MinutesPer
'Period': '0', #Quarter: 1,2,3,4,5,6,7,8,9,10
'PlayerExperience': '', # Rookie, Sophomore, Veteran
'PlayerPosition': '', # Center=C, Forward=F, Guard=G
'PlusMinus': 'N',
# 'PointDiff': '', # (Only for clutch endpoint) 5, 4, 3, 2, 1
'Rank': 'N',
'Season': '2022-23', # YYYY-YY, All Time for leagueleaders endpoint
'SeasonSegment': '', # Pre All-Star, Post All-Star
'SeasonType': 'Regular Season', # Season Type: Regular Season, Playoffs, Pre Season, All Star, PlayIn
'StatCategory': '', # (Only for leagueleaders endpoint) PTS, MIN, FGA, FGM, TOV, FG_PCT, FG3_PCT, FG3A, FG3M, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, BLK, STL, 
'ShotClockRange': '', # 24-22, 22-18 Very Early, 18-15 Very Early, 15-7 Average, 7-4 Late, 4-0 Very Late
'StarterBench': '', # Starters, Bench
'TeamID': '0',
'VsConference': '', # East, West
'VsDivision': '', # Central, Atlantic, Southeast, Southwest, Northwest, Pacific 
'Weight': '', # GT 200, LT 200, 
}

headers = {
'Accept': '*/*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9',
'Connection': 'keep-alive',
'Host': 'stats.nba.com',
'Origin': 'https://www.nba.com',
'Referer': 'https://www.nba.com/',
'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "Windows",
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

r = requests.request('GET', url=url, headers=headers, params=parameters)

In [104]:
r = r.json()

In [ ]:
n_group = 3
to_be_grouped = r['resultSets']['headers'][1]['columnNames'][6:]
grouped = []

for i in range(0, len(to_be_grouped), n_group):
    mini_groups = to_be_grouped[i:i+n_group]
    grouped.append(mini_groups)

grouped

In [ ]:
shotDistance = r['resultSets']['headers'][0]['columnNames']
stats_category = grouped[0]

In [153]:
new_column_index = [shotDistance, stats_category]
index = pd.MultiIndex.from_product(new_column_index, names=['shotDistance','category'])

In [175]:
index = [
    ("", 'PLAYER_ID'),
    ("", 'PLAYER_NAME'),
    ("", 'TEAM_ID'),
    ("", 'TEAM_ABBREVIATION'),
    ("", 'AGE'),
    ("", 'NICKNAME'),
    ('Less Than 5 ft.',    'FGM'),
    ('Less Than 5 ft.',    'FGA'),
    ('Less Than 5 ft.', 'FG_PCT'),
    (        '5-9 ft.',    'FGM'),
    (        '5-9 ft.',    'FGA'),
    (        '5-9 ft.', 'FG_PCT'),
    (      '10-14 ft.',    'FGM'),
    (      '10-14 ft.',    'FGA'),
    (      '10-14 ft.', 'FG_PCT'),
    (      '15-19 ft.',    'FGM'),
    (      '15-19 ft.',    'FGA'),
    (      '15-19 ft.', 'FG_PCT'),
    (      '20-24 ft.',    'FGM'),
    (      '20-24 ft.',    'FGA'),
    (      '20-24 ft.', 'FG_PCT'),
    (      '25-29 ft.',    'FGM'),
    (      '25-29 ft.',    'FGA'),
    (      '25-29 ft.', 'FG_PCT'),
    (      '30-34 ft.',    'FGM'),
    (      '30-34 ft.',    'FGA'),
    (      '30-34 ft.', 'FG_PCT'),
    (      '35-39 ft.',    'FGM'),
    (      '35-39 ft.',    'FGA'),
    (      '35-39 ft.', 'FG_PCT'),
    (        '40+ ft.',    'FGM'),
    (        '40+ ft.',    'FGA'),
    (        '40+ ft.', 'FG_PCT')
]

In [182]:
df = pd.DataFrame.from_records(r['resultSets']['rowSet'], columns=pd.MultiIndex.from_tuples(index))

In [183]:
df

\
    PLAYER_ID      PLAYER_NAME     TEAM_ID TEAM_ABBREVIATION   AGE NICKNAME   
0     1630639      A.J. Lawson  1610612742               DAL  22.0     A.J.   
1     1631260         AJ Green  1610612749               MIL  23.0       AJ   
2     1631100       AJ Griffin  1610612737               ATL  19.0       AJ   
3      203932     Aaron Gordon  1610612743               DEN  27.0    Aaron   
4     1628988    Aaron Holiday  1610612737               ATL  26.0    Aaron   
..        ...              ...         ...               ...   ...      ...   
534   1628380     Zach Collins  1610612759               SAS  25.0     Zach   
535    203897      Zach LaVine  1610612741               CHI  28.0     Zach   
536   1630192       Zeke Nnaji  1610612743               DEN  22.0     Zeke   
537   1630533  Ziaire Williams  1610612763               MEM  21.0   Ziaire   
538   1629627  Zion Williamson  1610612740               NOP  22.0     Zion   

    Less Than 5 ft.             5-9 ft.  ... 25-29 ft. 30-34 ft.              \
                FGM  FGA FG_PCT     FGM  ...    FG_PCT       FGM  FGA FG_PCT   
0                11   18  0.611     1.0  ...     0.750       0.0  0.0  0.000   
1                 3    6  0.500     0.0  ...     0.419       2.0  3.0  0.667   
2                50   80  0.625    33.0  ...     0.362       0.0  0.0  0.000   
3               326  456  0.715    19.0  ...     0.402       0.0  0.0  0.000   
4                32   62  0.516     6.0  ...     0.414       1.0  1.0  1.000   
..              ...  ...    ...     ...  ...       ...       ...  ...    ...   
534             154  234  0.658    46.0  ...     0.420       0.0  1.0  0.000   
535             309  469  0.659    33.0  ...     0.361       4.0  8.0  0.500   
536              90  124  0.726     2.0  ...     0.344       0.0  0.0  0.000   
537              39   56  0.696     7.0  ...     0.214       0.0  1.0  0.000   
538             252  374  0.674    20.0  ...     0.267       0.0  0.0  0.000   

    35-39 ft.             40+ ft.              
          FGM  FGA FG_PCT     FGM  FGA FG_PCT  
0         0.0  0.0    0.0     0.0  0.0    0.0  
1         0.0  0.0    0.0     0.0  0.0    0.0  
2         0.0  0.0    0.0     0.0  1.0    0.0  
3         0.0  1.0    0.0     0.0  4.0    0.0  
4         0.0  0.0    0.0     0.0  0.0    0.0  
..        ...  ...    ...     ...  ...    ...  
534       0.0  0.0    0.0     0.0  1.0    0.0  
535       0.0  3.0    0.0     0.0  0.0    0.0  
536       0.0  0.0    0.0     0.0  2.0    0.0  
537       0.0  0.0    0.0     1.0  1.0    1.0  
538       0.0  0.0    0.0     0.0  1.0    0.0  

[539 rows x 33 columns]